In [37]:
import pandas as pd

df = pd.read_csv('universal_mill_list.csv')

# Verify
print(f"Total rows: {len(df)}")
print(df.tail())

Total rows: 2032
                 X              Y   FID        UML_ID Group_Name  \
2027  1.116525e+07  152928.780816  2028  PO1000011910    UNKNOWN   
2028  1.138313e+07  -49294.319729  2029  PO1000011911    UNKNOWN   
2029  1.132490e+07  215410.869426  2030  PO1000011912    UNKNOWN   
2030  1.217452e+07   40106.205537  2031  PO1000011913    UNKNOWN   
2031  1.685138e+07 -638989.146403  2032  PO1000011914    UNKNOWN   

                      Parent_Com                   Mill_Name  \
2027           MAHATO INTI SAWIT           MAHATO INTI SAWIT   
2028  SAWIT JAYA MANDIRI LESTARI  SAWIT JAYA MANDIRI LESTARI   
2029       SAWIT RUPAT SEJAHTERA       SAWIT RUPAT SEJAHTERA   
2030                     GILFORD     GUNUNG RIJUAN SEJAHTERA   
2031          AGUMIL PHILIPPINES                      MAMUSI   

              RSPO_Statu RSPO_Type                    Date  ...   Longitude  \
2027  Not RSPO Certified            2021/07/14 00:00:00+00  ...  100.299131   
2028  Not RSPO Certified       

In [3]:
import sqlite3

In [38]:
# Create/connect to SQLite DB (creates if not exists)
conn = sqlite3.connect("bursa_palmai_database.db")  

# Write DataFrame to SQLite (creates a table named 'mth_prod_data')
df.to_sql(
    name="universal_mill_list",   # Table name
    con=conn,            # SQLite connection
    if_exists="replace", # Overwrite if table exists
    index=False          # Don't write DataFrame index
)

# Confirm
print("Table successfully created in SQLite!")

Table successfully created in SQLite!


In [34]:
query_result = pd.read_sql("SELECT * FROM mpob_stats limit 10", conn)
print(query_result)

                       ITEMS     MONTH_YEAR      VALUE
0  CRUDE PALM OIL PRODUCTION   October 2024  1797348.0
1  CRUDE PALM OIL PRODUCTION  November 2024  1621294.0
2  CRUDE PALM OIL PRODUCTION  December 2024  1486786.0
3  CRUDE PALM OIL PRODUCTION   January 2025  1237078.0
4  CRUDE PALM OIL PRODUCTION  February 2025  1188029.0
5  CRUDE PALM OIL PRODUCTION     March 2025  1387193.0
6  CRUDE PALM OIL PRODUCTION     April 2025  1685962.0
7      CRUDE PALM OIL STOCKS   October 2024   943472.0
8      CRUDE PALM OIL STOCKS  November 2024   907494.0
9      CRUDE PALM OIL STOCKS  December 2024   894321.0


In [39]:
import sqlite3

def list_tables(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Query SQLite's internal schema table
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    conn.close()
    return [table[0] for table in tables]

# Usage
tables = list_tables("bursa_palmai_database.db")
print("Tables in database:", tables)

Tables in database: ['company_mthly_prod', 'company_ext_rate', 'company_plt_area', 'mpob_stats', 'universal_mill_list']


In [30]:
def get_table_schema(db_path, table_name):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema = cursor.fetchall()
    conn.close()
    return schema

# Usage
print(get_table_schema("bursa_palmai_database.db", "company_plt_area"))

[(0, 'Company', 'TEXT', 0, None, 0), (1, 'Category', 'TEXT', 0, None, 0), (2, 'Year', 'INTEGER', 0, None, 0), (3, 'Value', 'INTEGER', 0, None, 0)]
